In [3]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [4]:
# 这是一个state schema
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    # add_message是一个reducer, 未指定reducer默认采用覆盖的策略
    # https://langchain-ai.github.io/langgraph/concepts/low_level/#default-reducer
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [5]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

# 加载 .env 文件
load_dotenv()

llm = init_chat_model("openai:gpt-4.1")

In [6]:
# 这是一个node，接受一个state作为参数，将聊天模型合并到一个简单的节点中
# State 中的 add_messages 函数将把 LLM 的响应消息附加到状态中已有的消息list中
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

In [7]:
# 添加一个 entry 点来告诉图表每次运行时从哪里开始工作
graph_builder.add_edge(START, "chatbot")
# 添加 exit 点，指示图表应在何处结束执行
graph_builder.add_edge("chatbot", END)

In [8]:
# 编译graph，在运行图表之前，我们需要编译它。这将创建一个 CompiledGraph ，我们可以用state调用它
graph = graph_builder.compile()

In [9]:
# 可视化图表
# 以使用 get_graph 方法和其中一种“draw”方法，例如 draw_ascii 或 draw_png 来可视化图形。每种 draw 方法都需要额外的依赖项
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [10]:
# 运行聊天机器人
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Goodbye!
